In [1]:
# Reading the bounding-box shapefile
import geopandas as gpd
path_to_border = "./Data/Border/Border.shp"
border = gpd.read_file(path_to_border)
timelapse="2023-01-01/2023-04-07"
collections = ["S2-16D-2"]
print(f"[INFO] Timelapse: {timelapse}")
print(f"[INFO] Collections: {collections}")


# Searching for the images that intersects with the border
import pystac_client
service = pystac_client.Client.open('https://data.inpe.br/bdc/stac/v1/')
item_search = service.search(
  collections=collections,
  datetime=timelapse,
  intersects=border.iloc[0].geometry
)
print(f"[INFO] Items matched: {item_search.matched()}")


# Defining which assets are to be read
# "NDVI" already downloaded in previous turn
assets = ["B02", "B03", "B04", "EVI"]
print(f"[INFO] Assets: {assets}")
print()


# Reading and writing the raster
import os
import rasterio
import datetime as dt
from rasterio.mask import mask
log_error = "./Data/Log.txt"

for i, item in enumerate(item_search.items()):
  bdc_tiles = item.properties['bdc:tiles'][0]
  datetime = item.id[-8:]
  print()
  print("x------------------x")
  print(f"[INFO] Item {i + 1} of {item_search.matched()}")
  print(f"[INFO] Item bdc:tiles: {bdc_tiles}")
  print(f"[INFO] Item datetime: {datetime}")

  for a in assets:
    print()
    print(f"[INFO] Asset: {a}")
    with rasterio.open(item.assets[a].href) as src:
      try:
        print(f"[DEBUG] Masking raster.")
        shapes = border.to_crs(src.crs)["geometry"]
        out_image, out_transform = mask(src, shapes, crop=True)
        out_meta = src.meta
        out_meta.update({
          "height": out_image.shape[1],
          "width": out_image.shape[2],
          "transform": out_transform
        })

        print(f"[DEBUG] Making new folder.")
        output_folder=f"./Data/Images/{bdc_tiles}/{datetime}"
        output_file=f"{output_folder}/{a}.tif"
        os.makedirs(output_folder, exist_ok=True)
        
        print(f"[DEBUG] Writing new raster in {output_file}...", end=" ")
        with rasterio.open(output_file, 'w', **out_meta) as dest:
          dest.write(out_image)
        print("done!")
      except Exception as e:
        print(f"[ERROR] {dt.datetime.now()}: '{e.__class__.__name__}: {e}'.")
        with open(log_error, "a") as log:
          log.write(f"[ERROR] {dt.datetime.now()}: {bdc_tiles} {datetime} '{e.__class__.__name__}: {e}'.\n")
        print(f"[INFO] Error appended to log file {log_error}.")

  print("x------------------x")
  print()


[INFO] Timelapse: 2023-01-01/2023-04-07
[INFO] Collections: ['S2-16D-2']
[INFO] Items matched: 49
[INFO] Assets: ['B02', 'B03', 'B04', 'EVI']


x------------------x
[INFO] Item 1 of 49
[INFO] Item bdc:tiles: 027031
[INFO] Item datetime: 20230407

[INFO] Asset: B02
[DEBUG] Masking raster.
[DEBUG] Making new folder.
[DEBUG] Writing new raster in ./Data/Images/027031/20230407/B02.tif... done!

[INFO] Asset: B03
[DEBUG] Masking raster.
[DEBUG] Making new folder.
[DEBUG] Writing new raster in ./Data/Images/027031/20230407/B03.tif... done!

[INFO] Asset: B04
[DEBUG] Masking raster.
[DEBUG] Making new folder.
[DEBUG] Writing new raster in ./Data/Images/027031/20230407/B04.tif... done!

[INFO] Asset: EVI
[DEBUG] Masking raster.
[DEBUG] Making new folder.
[DEBUG] Writing new raster in ./Data/Images/027031/20230407/EVI.tif... done!
x------------------x


x------------------x
[INFO] Item 2 of 49
[INFO] Item bdc:tiles: 027032
[INFO] Item datetime: 20230407

[INFO] Asset: B02
[DEBUG] Masking raster